# Decorators
---

### Introduction

The decorator's operation is based on wrapping the original function with a new "decorating" function (or class), hence the name "decoration". Of course, the decorating function does more, __because it can take the parameters of the decorated function and perform additional actions__ and that make it a real decorating function.

__NOTE__: Note that decorators, in Python, are still closures.

A decorator, apart from encapsulating a function, can also receive the arguments of that function and receive other arguments not related to it. The following are some of the possibilities with decorators.

Here are some case uses of decorators:
- Working with arguments:
    - Validation
    - Modification before passing to the decorated function
    - Return the result arguments parsed
- Debugging
    - Measurement of execution time.
    - Message logging.
- Thread synchronization.
- DRY (Don't Repeat Yourself) principle.
- Caching.

---

### Decorators: Only wrapping a function

This is the simplest example where the decorator encapsulates a function and does not need to send arguments to it (since it does not use them).

Here goes a code example:

In [10]:
def simple_decorator(func):  # The argument of a decorator it's always the wrapped function
    print(f"The simple_decorator is wrapping {func.__name__}")
    return func  # The last argument of a decorator is called
    # ^ It passes all the arguments send by the user even without concreting

@simple_decorator  # This can be interpreted as simple_decorator(simple_function())
def simple_function(arg1, arg2):  # This function now is received as an argument by the simple_decorator
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", "Bye")


The simple_decorator is wrapping simple_function
The parameters received are 'Hi' and 'Bye'


---

### Decorators: Working with the function arguments

One of the examples given above is that the decorator can work with arguments of the function it encapsulates, however, in the above demonstration we do not work with these arguments, so to do so we must use closures to access these arguments.

An example in code would be the following:

In [11]:
def simple_decorator(func):
    def wrapper(*args, **kwargs):  # Now i have access to the arguments sent by the user
        print(f"The simple_decorator is wrapping {func.__name__}")
        print(f"The decorator has access to {args[0]} and {args[1]}")  # We are using the *args parameters because in this example we haven't used keywords
        print(f"Argument that the decorated function doesn\'t use '{kwargs['decorator_argument']}'")
        return func(*args, **kwargs)  # In this section the encapsulated function must be called manually by sending the received arguments
    return wrapper  # We return the wrapper itself so that it receives the arguments sent to the function

@simple_decorator
def simple_function(arg1="", arg2="", **kwargs):
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", "Bye", decorator_argument="Yoh!")

The simple_decorator is wrapping simple_function
The decorator has access to Hi and Bye
Argument that the decorated function doesn't use 'Yoh!'
The parameters received are 'Hi' and 'Bye'



---

### Decorator: Receiving it's own arguments

The decorator can also receive arguments, which are not related to the function it encapsulates, but are used by the decorator itself. This is done by creating a function that returns the decorator, which in turn encapsulates the function.

Let's create an example that validates the datatype of the arguments received by the wrapped function:

In [20]:
def validate_datatype(validate_datatype):  # This example just adds an initial function that receive the specified decorator argument
    def simple_decorator(func):
        def wrapper(*args, **kwargs):
            print(f"The simple_decorator is wrapping {func.__name__}")
            print(f"The decorator has access to {args[0]} and {args[1]}\n")
            print(f"The decorator checks that the arguments received have the datatype '{str(validate_datatype)}':")
            for arg in args:
                print(f"The argument '{arg}' ", end="")
                if not isinstance(arg, validate_datatype):
                    print(f"doesn't have the expected datatype (has {str(type(arg))})")
                else:
                    print("have the expected datatype")
            print(f"\nArgument that the decorated function doesn\'t use '{kwargs['decorator_argument']}'")
            return func(*args, **kwargs)
        return wrapper
    return simple_decorator

@validate_datatype(int)  # The argument is passed directly from here
def simple_function(arg1="", arg2="", **kwargs):
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", 2, decorator_argument="Yoh!")

The simple_decorator is wrapping simple_function
The decorator has access to Hi and 2

The decorator checks that the arguments received have the datatype '<class 'int'>':
The argument 'Hi' doesn't have the expected datatype (has <class 'str'>)
The argument '2' have the expected datatype

Argument that the decorated function doesn't use 'Yoh!'
The parameters received are 'Hi' and '2'


---

### Stacking decorators

You can accumulate as many decorators as you wish, however, keep in mind that the order in which they are included indicates the order in which they will be executed.

The syntax for stacking decorators is as follows:

In [17]:
def outer_decorator(func):
    def wrapper():
        print("Outer time!")
        func()
    return wrapper


def inner_decorator(func):
    def wrapper():
        print("Inner time!")
        func()
    return wrapper

@outer_decorator
@inner_decorator
def func():
    print("Function time!")


func()
    

Outer time!
Inner time!
Function time!


Adding the decorators is the same as calling the function like this:

```outer_decorator(inner_decorator(func()))```

---

### Class decorators

Decorators, besides being defined in functions, can be created as classes by using a special method called ```__call__```.

Well, not only the special method mentioned above is taken into account, but also the constructor of the class has to be used.

The translation of each special method will be as follows:
- ```__init__```: It will be used as the first method of a decorator, that is to say, it is who the decorated function.
- ```__call__```: It is used as the wrapper method seen previously, that is to say, it will receive the arguments of the decorated function and will have access to the method of the previous method.

Before showing an example, __one of the advantages of using classes instead of functions__ to create decorators is that multiple methods can be generated to encapsulate the actions of the decorator and take advantage of the features provided by the inheritance of the class itself.

Now we enter an example, starting from this simpler decorator:

In [19]:
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("Decorator: Hi!")
        func(*args, **kwargs)
    return wrapper

@simple_decorator
def simple_function(arg1: str):
    print(f"Function received: {arg1}")


simple_function("Bye")

Decorator: Hi!
Function received: Bye


Let's say we want to pass it to a class, the translation would be as follows:

In [21]:
class SimpleDecorator:
    def __init__(self, func):  # Receive the function itself (like the function simple_decorator)
        self.func = func

    def __call__(self, *args, **kwargs):  # Receive the arguments of the function (like the function wrapper inside the simple_decorator)
        print("Decorator: Hi!")
        self.func(*args, **kwargs)  # Doesn't need to return anything

@SimpleDecorator
def simple_function(arg1: str):
    print(f"Function received: {arg1}")


simple_function("Bye")

Decorator: Hi!
Function received: Bye


---

### Class decorator with its own arguments

As with the function decorators, the class decorators can also receive arguments, which are not related to the function it encapsulates, but are used by the decorator itself.

It works in a similar way as with the functions, that is to say, you have to prepare the decorator so that the first argument it receives is its own arguments.

This at the class level involves using the constructor of the class to receive the arguments of the decorator and the __call__ method must use a closure to access the function and its arguments.

Adapting the previous example to include arguments would look something like this:


In [22]:
def attributes_decorator(arg: str):
    def simple_decorator(func):
        def wrapper(*args, **kwargs):
            print(f"Decorator received: {arg}")
            func(*args, **kwargs)
        return wrapper
    return simple_decorator

@attributes_decorator("Hi")
def simple_function(arg1: str):
    print(f"Function received: {arg1}")


simple_function("Bye")

Decorator received: Hi
Function received: Bye


And translated to class it would look something like this:

In [23]:
class AttributeDecorator:
    def __init__(self, arg: str):
        self.arg = arg

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print(f"Decorator received: {self.arg}")
            func(*args, **kwargs)
        return wrapper

@AttributeDecorator("Hi")
def simple_function(arg1: str):
    print(f"Function received: {arg1}")


simple_function("Bye")

Decorator received: Hi
Function received: Bye


---

### Classes decorated

As with functions, classes can also be decorated, however, the syntax is a bit different, since the decorator must be placed before the class definition.

The syntax would be as follows:

```python

def my_decorator():
    pass

@my_decorator
class MyClass:
    pass

obj = MyClass()

```

One use case for this is to create a decorator that counts how many times an attribute of the class is accessed.

This use case will look like this:




In [28]:
def count_use_attr(class_: type):  # The decorator receives the class decorated
    class_.new_class_method = class_.__getattribute__  # Copy, in a new function, the method to retrieve attributes
    
    def wrapper(self, arg_retrieved):  # The wrapper will know which attribute has been accessed
        if arg_retrieved == 'attr_1':
            print("The argument attr_1 have been accessed")
        return class_.new_class_method(self, arg_retrieved) 
        # ^ Pass to the method to retrieve attributes the attr asked to return the control to the class

    class_.__getattribute__ = wrapper   # Now change the method to retrieve arguments to the own wrapper
    return class_

@count_use_attr
class DummyClass:
    def __init__(self, attr_1, attr_2):
        self.attr_1 = attr_1
        self.attr_2 = attr_2


obj = DummyClass(1, 2)
print(obj.attr_1)  # wrapper will retrieve attr_1
print(obj.attr_2)  # wrapper will retrieve attr_2
print(obj.attr_1)  # wrapper will retrieve attr_1


The argument attr_1 have been accessed
1
2
The argument attr_1 have been accessed
1
